In [1]:
!pip install youtube-transcript-api



   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 1.0 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.2 MB 1.0 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 1.1 MB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.2 MB 1.1 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

In [2]:
from langchain_google_vertexai import VertexAIEmbeddings ,ChatVertexAI
embedding = VertexAIEmbeddings(
    model_name= "text-embedding-005",
    project="modular-shard-456912-m4",     
    location="us-central1"
)

In [3]:
llm = ChatVertexAI(model="gemini-1.5-pro-002", project="modular-shard-456912-m4",
    location="us-central1")

In [4]:
video_id = "kQmXtrmQ5Zg"
try:
     transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

# Flatten it to plain text
     transcript = " ".join(chunk["text"] for chunk in transcript_list)
     print(transcript)
except TranscriptsDisabled:
    print("No captions available for this video.")

[Music] hey everyone hello thank you all for coming uh my name is mahes and I'm on the applied AI team at anthropic um really excited to see a very full room and very excited that you chose me over open AI thank you very much so today we're going to be talking about mCP model context protocol um this is more of a talk than a workshop but I'll do my best to keep it interactive if you want to ask questions uh feel free to do so and I'll do my best to answer them um today we're going to talk about the philosophy behind mCP and why we at anthropic thought that it was an important thing to launch and build uh we're going to talk about some of the traction about uh of mCP in the last couple of months um and then some of the patterns that allow mCP to be adopted for AI applications for agents um and then the road map and where we're going from here cool so our motivation behind mCP was the core concept that models are only as good as the context we provide to them um this is a pretty obvious 

In [5]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap =200 )
docs = spliter.create_documents([transcript])

In [6]:
print(len(docs))

106


In [7]:
vector_store = FAISS.from_documents(docs,embedding)

In [8]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [15]:
retriever.invoke('What is AI')

[Document(id='6c48d882-1bdd-4668-98c8-3af22aa81397', metadata={}, page_content="it kind of comes up in almost every anthropic conversation with uh people that we work with and a lot of our customers um this this slide covers a few different personas um but we can start with the the applications and the idees um this has been really exciting recently and it provides this really nice way for people that are coding in IDE to provide context to that ID while they're working um and the Agents inside those IDs go and talk to these external systems uh like GitHub like documentation sites Etc we've also seen a lot of development on the the server side um I think to date there are something like 1100 Community built servers uh that folks have built um and published open source there are also a bunch of servers built by companies themselves I just built one as an example uh there are folks like uh and a bunch of others that have published official Integrations for ways to hook into their systems

In [9]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [10]:
question          = "Did the talk about the making connection with data base with the help OF MCP in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [16]:
retrieved_docs[0].page_content

"you just always call that Vector DB uh sorry second so the most things NCP is able to more authentication how I'm going to get to that one later um because it's very relevant and we have a lot to say so I may have miss this on that um but so have you gone down the route of using a gentle framework tool calling um as you progressed would you just WRA the mCP with a tool if you had a Sy yeah I think that it sounds like the broader question is how does mCP fit in with agent Frameworks um cool yeah I mean the aners they they kind of complement each other um actually Lang graph just this week released a bunch of connectors for or I think they're called adapters for langra agen to connect to mCP so if you already have a system built inside Lang graph or another agent framework uh if it has this connector to mCP servers you can expose those servers to the agent uh without having to change your system itself as long as that adapter is installed so we don't think mCP is going to replace agent"

In [12]:
content_text = "/n/n".join(doc.page_content for doc in retrieved_docs )
content_text

"you just always call that Vector DB uh sorry second so the most things NCP is able to more authentication how I'm going to get to that one later um because it's very relevant and we have a lot to say so I may have miss this on that um but so have you gone down the route of using a gentle framework tool calling um as you progressed would you just WRA the mCP with a tool if you had a Sy yeah I think that it sounds like the broader question is how does mCP fit in with agent Frameworks um cool yeah I mean the aners they they kind of complement each other um actually Lang graph just this week released a bunch of connectors for or I think they're called adapters for langra agen to connect to mCP so if you already have a system built inside Lang graph or another agent framework uh if it has this connector to mCP servers you can expose those servers to the agent uh without having to change your system itself as long as that adapter is installed so we don't think mCP is going to replace agent/

In [13]:
final_prompt = prompt.invoke({"context": content_text, "question": question})
print(final_prompt)

text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      you just always call that Vector DB uh sorry second so the most things NCP is able to more authentication how I'm going to get to that one later um because it's very relevant and we have a lot to say so I may have miss this on that um but so have you gone down the route of using a gentle framework tool calling um as you progressed would you just WRA the mCP with a tool if you had a Sy yeah I think that it sounds like the broader question is how does mCP fit in with agent Frameworks um cool yeah I mean the aners they they kind of complement each other um actually Lang graph just this week released a bunch of connectors for or I think they're called adapters for langra agen to connect to mCP so if you already have a system built inside Lang graph or another agent framework uh if it has this connector to mCP servers yo

In [14]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the speaker discussed connecting to databases with MCP.  They explained that an MCP server can act as a wrapper or a way of federating access to various systems and tools, including databases, allowing LLMs to query and fetch data.  This allows LLMs to access and utilize data stored in databases without the agent builder having to focus on the server or data itself.  They gave the example of a CRM like Salesforce, where you might want to read and write data hosted on a remote server.  They also mentioned that MCP can connect to things on a local system, like version control with Git.



## BULDING CHAIN